In [20]:
# -------
# IMPORT LIBRAIRIES
# -------
import numpy as np
import pandas as pd
import plotly.express as px
from python_module import sabr
from python_module import blackscholes
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from scipy.cluster.hierarchy import linkage, fcluster

# -------
# DISPLAY SETTINGS
# -------
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.options.display.float_format = lambda x: f'{x:_.4f}'

In [21]:
# -------
# GENERATE MARKET DATA & PORTFOLIO
# -------
S = F = 100
alpha = 0.2
beta = 1
rho = -0.4
nu = 0.5
market_data_list = list()
for time_to_maturity in np.linspace(start=1, stop=250, num=10, dtype=int):
    T = time_to_maturity / 250
    for K in np.linspace(start=80, stop=100, num=10):
        IV = sabr.compute_vol(F, K, T, alpha, beta, rho, nu)
        pricing_results = blackscholes.compute_option(S, K, T, 0, IV, 'put', True)
        market_data_list.append({'time_to_maturity': time_to_maturity, 'T': T, 'K': K, 'IV': IV, **pricing_results})
market_data_df = pd.DataFrame(market_data_list)
market_data_df['weights'] = abs(np.random.normal(size=market_data_df.shape[0]))

In [22]:
market_data_df.tail()

,time_to_maturity,T,K,IV,price,delta,gamma,vega,theta,vanna,volga,weights
95,250,1.0000,91.1111,0.2117,4.3751,-0.2927,0.0162,0.3438,-0.0144,-0.5419,29.5622,2.6022
96,250,1.0000,93.3333,0.2088,5.1372,-0.3318,0.0174,0.3630,-0.0150,-0.3929,17.0855,0.2199
97,250,1.0000,95.5556,0.2060,5.9948,-0.3731,0.0184,0.3786,-0.0155,-0.2161,6.9943,0.1329
98,250,1.0000,97.7778,0.2035,6.9522,-0.4160,0.0192,0.3901,-0.0157,-0.0166,0.3529,0.8095
99,250,1.0000,100.0000,0.2012,8.0119,-0.4599,0.0197,0.3969,-0.0158,0.1985,-1.9962,1.1988


In [23]:
market_data_df['delta'] *= -1

In [24]:
# -------
# FEATURE ENGINEERING
# -------
market_data_df['log_K'] = np.log(market_data_df['K']/S)
market_data_df['exp_t'] = np.log(market_data_df['T'])
features = ['log_K', 'exp_t']

# -------
# MODEL FIT
# -------
degree = 2
poly_features = PolynomialFeatures(degree=degree)
X = market_data_df[features]
X_poly = poly_features.fit_transform(X)
Y = market_data_df['price']
model = LinearRegression(fit_intercept=False)
model.fit(X_poly, Y)

# -------
# MODEL PREDICTION
# -------
X = pd.DataFrame(X_poly).multiply(model.coef_)
Z = linkage(X, method='complete', metric='euclidean')
num_clusters = 10
clusters = fcluster(Z, num_clusters, criterion='maxclust')
market_data_df['cluster'] = [str(x) for x in clusters]

In [25]:
px.scatter(data_frame=market_data_df, x='K', y='time_to_maturity', color='cluster', size='delta')